In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler 
from sklearn import metrics 
np.random.seed(0)

In [2]:
datos=pd.read_csv("SCADA_data_WT2337.csv")

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.DataFrame({'date_time': datos['date_time'] ,'TempEjeLento_1': datos['TempEjeLento_1'], 'TempAmbMean': datos['TempAmbMean'], 'TempRodamMultipMean': datos['TempRodamMultipMean'], 'TempCojLAMean': datos['TempCojLAMean'],'TempCojLOAMean': datos['TempCojLOAMean'], 'TempGenMean': datos['TempGenMean'], 'PotMean': datos['PotMean'], 'VelRotorMean': datos['VelRotorMean']})
cols = df.columns.tolist()

In [5]:
#PotMean
df['PotMean'][df['PotMean']< 0] = None
df['PotMean'][df['PotMean']> 2000] = None
#TempAmbMean
df['TempAmbMean'][df['TempAmbMean']< -5] = None
df['TempAmbMean'][df['TempAmbMean']> 40] = None
#TempCojLAMean
df['TempCojLAMean'][df['TempCojLAMean']< 0] = None
df['TempCojLAMean'][df['TempCojLAMean']> 120] = None
#TempCojLOAMean
df['TempCojLOAMean'][df['TempCojLOAMean']< 0] = None
df['TempCojLOAMean'][df['TempCojLOAMean']> 120] = None
#TempEjeLento_1
df['TempEjeLento_1'][df['TempEjeLento_1']< 0] = None
df['TempEjeLento_1'][df['TempEjeLento_1']> 120] = None
#TempGenMean
df['TempGenMean'][df['TempGenMean']< 0] = None
df['TempGenMean'][df['TempGenMean']> 175] = None
#TempRodamMultipMean
df['TempRodamMultipMean'][df['TempRodamMultipMean']< 0] = None
df['TempRodamMultipMean'][df['TempRodamMultipMean']> 120] = None
#VelRotorMean
df['VelRotorMean'][df['VelRotorMean']< 0] = None
df['VelRotorMean'][df['VelRotorMean']> 50] = None

In [6]:
df['TempAmbMean']=df['TempAmbMean'].interpolate(method='pchip', order=3, limit_area='inside')
df['TempAmbMean']=df['TempAmbMean'].fillna(method='backfill')
df['TempAmbMean']=df['TempAmbMean'].fillna(method='ffill')

df['TempEjeLento_1']=df['TempEjeLento_1'].interpolate(method='pchip', order=3)
df['TempEjeLento_1']=df['TempEjeLento_1'].fillna(method='backfill')

df['TempRodamMultipMean']=df['TempRodamMultipMean'].interpolate(method='pchip', order=3)
df['TempRodamMultipMean']=df['TempRodamMultipMean'].fillna(method='backfill')

df['TempCojLAMean']=df['TempCojLAMean'].interpolate(method='pchip', order=3)
df['TempCojLAMean']=df['TempCojLAMean'].fillna(method='backfill')

df['TempCojLOAMean']=df['TempCojLOAMean'].interpolate(method='pchip', order=3)
df['TempCojLOAMean']=df['TempCojLOAMean'].fillna(method='backfill')

df['TempGenMean']=df['TempGenMean'].interpolate(method='pchip', order=3)
df['TempGenMean']=df['TempGenMean'].fillna(method='backfill')

df['PotMean']=df['PotMean'].interpolate(method='pchip', order=3)
df['PotMean']=df['PotMean'].fillna(method='backfill')

df['VelRotorMean']=df['VelRotorMean'].interpolate(method='pchip', order=3)
df['VelRotorMean']=df['VelRotorMean'].fillna(method='backfill')

df['TempEjeLento_1']=df['TempEjeLento_1'].fillna(method='ffill')
df['TempRodamMultipMean']=df['TempRodamMultipMean'].fillna(method='ffill')
df['TempCojLAMean']=df['TempCojLAMean'].fillna(method='ffill')
df['TempCojLOAMean']=df['TempCojLOAMean'].fillna(method='ffill')
df['TempGenMean']=df['TempGenMean'].fillna(method='ffill')
df['PotMean']=df['PotMean'].fillna(method='ffill')
df['VelRotorMean']=df['VelRotorMean'].fillna(method='ffill')

In [7]:
df.reset_index(drop=True, inplace=True)

In [8]:
datos_listos=df.copy()

In [9]:
datos_t=pd.DataFrame({ 'TempEjeLento_1':df['TempEjeLento_1'],'date_time': df['date_time'] ,})
datos_t.drop([0], inplace=True)
datos_t.reset_index(drop=True, inplace=True)
datos_t

,TempEjeLento_1,date_time
0,1.999570e+01,2013-01-01 00:10:00.0
1,1.999570e+01,2013-01-01 00:20:00.0
2,1.999570e+01,2013-01-01 00:30:00.0
3,1.999570e+01,2013-01-01 00:40:00.0
4,1.999570e+01,2013-01-01 00:50:00.0
...,...,...
358128,8.684532e+08,2019-10-28 00:10:00.0
358129,8.685083e+08,2019-10-28 00:20:00.0
358130,8.685633e+08,2019-10-28 00:30:00.0
358131,8.686183e+08,2019-10-28 00:40:00.0


In [10]:
datos_tmenos1=pd.DataFrame({'TempEjeLento_1-1': df['TempEjeLento_1'],'TempAmbMean-1': df['TempAmbMean'], 'TempRodamMultipMean-1': df['TempRodamMultipMean'],'TempCojLOAMean-1': df['TempCojLOAMean'], 'TempGenMean-1': df['TempGenMean'], 'PotMean-1': df['PotMean'], 'VelRotorMean-1': df['VelRotorMean']})
datos_tmenos1.drop([len(df['VelRotorMean'])-1], inplace=True)
datos_tmenos1.reset_index(drop=True, inplace=True)
datos_tmenos1

,TempEjeLento_1-1,TempAmbMean-1,TempRodamMultipMean-1,TempCojLOAMean-1,TempGenMean-1,PotMean-1,VelRotorMean-1
0,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639
1,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639
2,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639
3,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639
4,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639
...,...,...,...,...,...,...,...
358128,8.683982e+08,18.02380,57.0572,36.1502,50.8232,153.996,11.7559
358129,8.684532e+08,17.96310,58.0939,36.2752,51.2622,190.546,12.1454
358130,8.685083e+08,17.90240,59.1305,36.4003,51.7013,190.080,12.1656
358131,8.685633e+08,17.84170,60.1671,36.5253,52.1404,164.409,11.9320


In [11]:
datos_listos=pd.DataFrame({'date_time': datos_t['date_time'], 'TempEjeLento_1-1': datos_tmenos1['TempEjeLento_1-1'],'TempAmbMean-1': datos_tmenos1['TempAmbMean-1'],'TempRodamMultipMean-1': datos_tmenos1['TempRodamMultipMean-1'],'TempCojLOAMean-1': datos_tmenos1['TempCojLOAMean-1'],'TempGenMean-1': datos_tmenos1['TempGenMean-1'], 'PotMean-1': datos_tmenos1['PotMean-1'], 'VelRotorMean-1': datos_tmenos1['VelRotorMean-1'],'TempEjeLento_1': datos_t['TempEjeLento_1']})
datos_listos

,date_time,TempEjeLento_1-1,TempAmbMean-1,TempRodamMultipMean-1,TempCojLOAMean-1,TempGenMean-1,PotMean-1,VelRotorMean-1,TempEjeLento_1
0,2013-01-01 00:10:00.0,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639,1.999570e+01
1,2013-01-01 00:20:00.0,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639,1.999570e+01
2,2013-01-01 00:30:00.0,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639,1.999570e+01
3,2013-01-01 00:40:00.0,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639,1.999570e+01
4,2013-01-01 00:50:00.0,1.999570e+01,2.39048,64.1288,18.8788,38.4616,229.510,12.1639,1.999570e+01
...,...,...,...,...,...,...,...,...,...
358128,2019-10-28 00:10:00.0,8.683982e+08,18.02380,57.0572,36.1502,50.8232,153.996,11.7559,8.684532e+08
358129,2019-10-28 00:20:00.0,8.684532e+08,17.96310,58.0939,36.2752,51.2622,190.546,12.1454,8.685083e+08
358130,2019-10-28 00:30:00.0,8.685083e+08,17.90240,59.1305,36.4003,51.7013,190.080,12.1656,8.685633e+08
358131,2019-10-28 00:40:00.0,8.685633e+08,17.84170,60.1671,36.5253,52.1404,164.409,11.9320,8.686183e+08


In [12]:
from datetime import datetime,timedelta
datos_listos['date_time']=pd.to_datetime(datos_listos['date_time'])

In [13]:
data_tv=datos_listos.copy()
mask = ((data_tv['date_time'] >= '2017-02-06 00:00:00') & (data_tv['date_time'] < '2018-01-01 00:00:00') ) 
train_tv=data_tv.loc[mask]
train_tv.reset_index(drop=True, inplace=True)

In [14]:
longitud=len(train_tv)
#longitud=longitud*70/100
longitud=(longitud)/144
longitud=longitud*90/100
longitud=round(longitud)
longitud=longitud*144
longitud

42624

In [15]:
training=train_tv.loc[0:round(longitud)-1]
training.reset_index(drop=True, inplace=True)
training

,date_time,TempEjeLento_1-1,TempAmbMean-1,TempRodamMultipMean-1,TempCojLOAMean-1,TempGenMean-1,PotMean-1,VelRotorMean-1,TempEjeLento_1
0,2017-02-06 00:00:00,22.879502,7.43394,73.7686,35.0659,82.3580,1502.760,17.7552,22.880098
1,2017-02-06 00:10:00,22.880098,7.39546,73.7262,34.9930,83.0070,1521.620,17.0368,22.880692
2,2017-02-06 00:20:00,22.880692,7.35698,73.6839,34.9201,83.6560,1512.130,17.3090,22.881285
3,2017-02-06 00:30:00,22.881285,7.31849,73.6415,34.8472,84.3050,1488.260,16.8005,22.881875
4,2017-02-06 00:40:00,22.881875,7.28001,73.5991,34.7743,84.9540,1494.730,16.6559,22.882463
...,...,...,...,...,...,...,...,...,...
42619,2017-11-28 23:10:00,22.229200,10.87230,72.8500,29.0079,50.6938,532.308,15.5693,22.365700
42620,2017-11-28 23:20:00,22.365700,10.87490,72.7006,29.0459,51.2225,538.059,15.8019,22.493200
42621,2017-11-28 23:30:00,22.493200,10.87750,72.4397,29.0839,51.7513,655.092,16.6070,22.582200
42622,2017-11-28 23:40:00,22.582200,10.88000,72.1787,29.1219,52.2800,724.791,17.0285,22.666800


In [16]:
validation=train_tv.loc[round(longitud):len(train_tv)]
validation.reset_index(drop=True, inplace=True)
validation

,date_time,TempEjeLento_1-1,TempAmbMean-1,TempRodamMultipMean-1,TempCojLOAMean-1,TempGenMean-1,PotMean-1,VelRotorMean-1,TempEjeLento_1
0,2017-11-29 00:00:00,22.7514,10.88510,71.6569,29.1979,53.3376,731.328000,17.103700,22.8360
1,2017-11-29 00:10:00,22.8360,10.88770,71.3959,29.1589,53.8663,743.456000,16.813000,22.9152
2,2017-11-29 00:20:00,22.9152,10.89020,71.1350,29.1927,54.3951,487.361000,15.450100,22.9417
3,2017-11-29 00:30:00,22.9417,10.89280,70.8741,29.2266,54.7146,603.615000,16.145600,22.9549
4,2017-11-29 00:40:00,22.9549,10.89540,70.6131,29.2604,54.8153,551.536000,15.615300,22.9682
...,...,...,...,...,...,...,...,...,...
4747,2017-12-31 23:10:00,20.2618,8.34259,58.4853,31.6632,79.7916,83.266200,2.821730,20.1293
4748,2017-12-31 23:20:00,20.1293,8.27274,56.0265,31.4196,78.4111,0.044976,0.367458,19.8581
4749,2017-12-31 23:30:00,19.8581,8.20288,53.5842,31.1759,77.0307,0.094435,0.481486,19.5864
4750,2017-12-31 23:40:00,19.5864,8.13303,51.1418,30.9322,75.6502,0.143894,0.275624,19.3147


In [17]:
test=datos_listos.copy()
mask = ((test['date_time'] >= '2018-01-01 00:00:00') & (test['date_time'] < '2018-12-01 00:00:00')) 
test=test.loc[mask]
test.reset_index(drop=True, inplace=True)
test

,date_time,TempEjeLento_1-1,TempAmbMean-1,TempRodamMultipMean-1,TempCojLOAMean-1,TempGenMean-1,PotMean-1,VelRotorMean-1,TempEjeLento_1
0,2018-01-01 00:00:00,19.0430,7.99331,46.8007,30.4449,72.8892,0.242812,0.231295,18.7970
1,2018-01-01 00:10:00,18.7970,7.92346,46.2656,30.2012,71.5087,0.292271,0.332392,18.6419
2,2018-01-01 00:20:00,18.6419,7.85360,45.7959,29.9576,70.1282,0.341730,0.379906,18.4947
3,2018-01-01 00:30:00,18.4947,7.78375,45.3262,29.7139,68.7477,0.391189,0.000000,18.3476
4,2018-01-01 00:40:00,18.3476,7.71389,44.8565,29.4703,67.3672,0.440648,0.000000,18.2004
...,...,...,...,...,...,...,...,...,...
48091,2018-11-30 23:10:00,18.4543,6.04839,52.5553,23.4519,42.6439,59.172800,11.642600,18.4170
48092,2018-11-30 23:20:00,18.4170,5.99709,52.3561,23.4182,42.6022,73.653900,11.672200,18.3798
48093,2018-11-30 23:30:00,18.3798,5.94744,52.1570,23.3844,42.5605,62.996200,11.703200,18.3346
48094,2018-11-30 23:40:00,18.3346,5.92394,51.9578,23.3507,42.5188,45.085900,11.694900,18.2666


In [18]:
from numpy import array
from numpy import hstack

def datosstack(data):
    
        X1 = array(data['TempAmbMean-1'])
        X2 = array(data['TempRodamMultipMean-1'])
        #X3 = array(data['TempCojLAMean-1'])
        X3 = array(data['TempCojLOAMean-1'])
        X4 = array(data['TempGenMean-1'])
        X5 = array(data['PotMean-1'])
        X6 = array(data['VelRotorMean-1'])
        X7 = array(data['TempEjeLento_1-1'])
        X8 = array(data['TempEjeLento_1'])

        X1 = X1.reshape((len(X1), 1))
        X2 = X2.reshape((len(X2), 1))
        X3 = X3.reshape((len(X3), 1))
        X4 = X4.reshape((len(X4), 1))
        X5 = X5.reshape((len(X5), 1))
        X6 = X6.reshape((len(X6), 1))
        X7 = X7.reshape((len(X7), 1))
        X8 = X8.reshape((len(X8), 1))
        #X9 = X8.reshape((len(X9), 1))
        
        dataset = hstack((X1, X2, X3, X4, X5, X6, X7, X8))            

        return dataset 

In [19]:
dataset_train=datosstack(training)
dataset_validation=datosstack(validation)
dataset_test=datosstack(test)

dataset_train.shape

(42624, 8)

In [20]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        #end_ix = n_steps*i + n_steps
        end_ix = i + n_steps
        #start_ix = end_ix - n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
            # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [21]:
n_steps = 144
xtrain, ytrain = split_sequences(dataset_train, n_steps)
print(xtrain.shape, ytrain.shape)

(42481, 144, 7) (42481,)


In [22]:
xvalidation, yvalidation = split_sequences(dataset_validation, n_steps)
print(xvalidation.shape, yvalidation.shape)

(4609, 144, 7) (4609,)


In [23]:
xtest, ytest = split_sequences(dataset_test, n_steps)
print(xtest.shape, ytest.shape)

(47953, 144, 7) (47953,)


In [24]:
from tensorflow import keras
from tensorflow.keras import layers

In [25]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [26]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [27]:
input_shape = xtrain.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.2,
)

model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3)
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    xtrain,
    ytrain,
    validation_split=0.2,
    epochs=200,
    batch_size=128,
    callbacks=callbacks,
)

model.evaluate(xvalidation, yvalidation, verbose=1)

Epoch 1/200
266/266 [==============================] - 75s 190ms/step - loss: 1164.7209 - val_loss: 23.5152
Epoch 2/200
266/266 [==============================] - 51s 191ms/step - loss: 69.8246 - val_loss: 12.4553
Epoch 3/200
266/266 [==============================] - 51s 191ms/step - loss: 56.9228 - val_loss: 9.3187
Epoch 4/200
266/266 [==============================] - 49s 186ms/step - loss: 47.2294 - val_loss: 5.0384
Epoch 5/200
266/266 [==============================] - 56s 210ms/step - loss: 43.0938 - val_loss: 4.3435
Epoch 6/200
266/266 [==============================] - 51s 191ms/step - loss: 37.4126 - val_loss: 4.8076
Epoch 7/200
266/266 [==============================] - 50s 189ms/step - loss: 35.6718 - val_loss: 4.9383
Epoch 8/200
266/266 [==============================] - 50s 188ms/step - loss: 33.1874 - val_loss: 5.9003
Epoch 9/200
266/266 [==============================] - 50s 188ms/step - loss: 32.1420 - val_loss: 6.0494
Epoch 10/200
266/266 [=============================

6.366038799285889

In [28]:
model.save("test3_transformer.h5")